<img src="https://files.cdn.thinkific.com/file_uploads/133292/images/d57/1b0/1a9/P03.jpg">

`MCSDSC02P03V3______`

`MCSDSELIMC02P03V2250621`

`MCSDSELIMC02P03V1130621`

<h3><font color="#89DD00">Комментарий ассистента</font></h3>

Супер! Снова проект принят с первого раза! :)

## Постановка задачи

Заказчик исследования: необходимо выяснить, как разные страховые случаи, возраст клиента и его доход влияют на статус выплаты страховой компенсации. Для этого необходимо:
1. Импортировать библиотеки и таблицу с данными
2. Получить общую информацию о данных
3. Обработать данные в таблице:
    * заполнить/заменить/удалить пропуски в таблице
    * обработать/исправить типы данных
    * обработать дубликаты и привести все к одному типу и виду
4. Провести необходимые расчеты, по результатам которых можно будет ответить на следующие вопросы:
    * есть ли зависимость между возрастной группой и статусом выплаты?
    * есть ли зависимость между причиной страхового случая и статусом выплаты?
    * есть ли зависимость между доходом клиента и статусом выплаты?

## Импорт данных и библиотек

Импортируем библиотеки, прочитаем файл `insurance.csv` и получим общую информацию о данных.

In [626]:
import pandas as pd
import numpy as np
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem

data = pd.read_csv('../../data/insurance.csv')
data

,id,perc_of_compensation_paid,age_in_days,age_id,age_group,income,count_3-6_months_late,count_6-12_months_late,count_more_than_12_months_late,cause
0,649,0.001,27384.0,4,Пожилой,NaN,0,0,0,повреждение автомобиля
1,81136,0.124,23735.0,4,пожилой,285140.0,0,0,0,упало дерево на автомобиль
2,70762,1.000,17170.0,3,Зрелый,-186030.0,0,0,0,сломала ногу в отпуске
3,53935,0.198,16068.0,3,Зрелый,-123540.0,0,0,0,хищение автомобиля (каско)
4,15476,0.041,10591.0,2,Молодой,-200020.0,1,0,0,несчастный случай в отпуске
...,...,...,...,...,...,...,...,...,...,...
2296,109416,0.413,17525.0,3,Зрелый,NaN,0,0,0,кража в квартире
2297,4268,0.109,21906.0,4,Пожилой,NaN,0,0,0,проникли в жилье
2298,85708,0.036,25925.0,4,Пожилой,NaN,0,0,0,обокрали дачу
2299,105186,0.010,23370.0,4,Пожилой,NaN,0,0,0,повреждение автомобиля


Столбцы содержат следующую информацию:

* id — ID держателя полиса
* perc_of_compensation_paid — процент выплаченной компенсации
* age_in_days — возраст держателя полиса в днях
* age_id — ID возрастной группы
* age_group — возрастная группа 
* income — доход держателя полиса
* count_3-6_months_late — полис оформлен не позднее 3-6 месяцев
* count_6-12_months_late — полис оформлен не позднее 6-12 месяцев
* count_more_than_12_months_late — полис оформлен больше года назад
* cause — причина страхового случая

### Выводы

* таблица содержит 2301 струку и 10 колонок
* некоторые значения в столбце "income" имеют значение NaN

## Предобработка данных

**Обработка пропусков**

Проверим данные на наличие пропусков, просуммировав их.

In [627]:
data.isnull().sum()

id                                 0
perc_of_compensation_paid          0
age_in_days                        0
age_id                             0
age_group                          0
income                            65
count_3-6_months_late              0
count_6-12_months_late             0
count_more_than_12_months_late     0
cause                              0
dtype: int64

Заполним пропуски в столбце `income`.

In [628]:
data['income'] = data['income'].fillna(0)

Проверяем, что пропусков в `income` не осталось.

In [629]:
data.isnull().sum()

id                                0
perc_of_compensation_paid         0
age_in_days                       0
age_id                            0
age_group                         0
income                            0
count_3-6_months_late             0
count_6-12_months_late            0
count_more_than_12_months_late    0
cause                             0
dtype: int64

**Замена типа данных**

Проверяем, что в столбцах `age_in_days` и `income` вещественный тип данных.

In [630]:
data.dtypes

id                                  int64
perc_of_compensation_paid         float64
age_in_days                       float64
age_id                              int64
age_group                          object
income                            float64
count_3-6_months_late               int64
count_6-12_months_late              int64
count_more_than_12_months_late      int64
cause                              object
dtype: object

Для этих столбцов нужен целочисленный тип, поэтому переведем их, а также избавимся от отрицательных значений в `income`.

In [631]:
data = data.astype({'income': 'int64', 'age_in_days': 'int64'})

In [632]:
data.dtypes

id                                  int64
perc_of_compensation_paid         float64
age_in_days                         int64
age_id                              int64
age_group                          object
income                              int64
count_3-6_months_late               int64
count_6-12_months_late              int64
count_more_than_12_months_late      int64
cause                              object
dtype: object

In [633]:
data['income'] = abs(data['income'])

**Поиск дубликатов с учетом регистра**

Проверим уникальные категории в столбце `age_group`.

In [634]:
data['age_group'].unique()

array(['Пожилой', 'пожилой', 'Зрелый', 'Молодой', 'ЗРЕЛЫЙ', 'ПОЖИЛОЙ',
       'зрелый', 'молодой', 'МОЛОДОЙ'], dtype=object)

Приведем их к нижнему регистру.

In [635]:
data['age_group'] = data['age_group'].str.lower()

**Обработка дубликатов**

Установим количество явных дубликатов. Если найдутся, то удалим и снова проверим, что их не осталось.

In [636]:
data.duplicated().sum()

7

In [637]:
data = data.drop_duplicates().reset_index(drop=True)

In [638]:
data.duplicated().sum()

0

**Стэмминг**

Посчитаем уникальные значения в столбце `cause`.

In [639]:
data['cause'].value_counts()

кража автомобиля               112
травма во время отдыха         107
сломала ногу в отпуске          95
упало дерево на автомобиль      94
украли багаж в отпуске          94
травма в отпуске                94
сгорел дом                      93
замкнуло электричество дома     91
сгорела квартира                91
врезались в автомобиль          91
несчастный случай в отпуске     91
сотрясение во время отдыха      90
проникли в жилье                89
повреждение автомобиля          89
потоп в квартире                88
хищение автомобиля (каско)      88
несчастный случай на отдыхе     85
повреждение авто                85
кража авто                      82
обокрали дачу                   81
поломка авто                    81
кража в квартире                79
украли авто                     78
взломали квартиру               77
дтп                             76
затопило дом                    73
Name: cause, dtype: int64

Выделим очевидные разные стеммы:
1. автомобиль/дтп
2. отдых/отпуск
3. дом/жилье/дача/квартира

Из них можно выделить следующие виды страхования:
1. проишествия во время отдыха
2. случаи с авто
3. случаи с недвижимостью

Напишем функцию, которая принимает неизменяемую часть слова и возвращает все названия категорий с ним. Сначала попробуем стемминг и если не поймаем «чужих» названий — значит этот метод подойдет.

In [640]:
russian_stemmer = SnowballStemmer(language='russian')

def category(stem_word):
    data_stem = data['cause'].drop_duplicates() # удаление дубликатов случаев
    case_list = []

    for cat in data_stem: # перебор по категориям
        for word in cat.split(): # перебор по словам в категориях
            stemmed_word = russian_stemmer.stem(word) # поиск основы заданного слова
            if stemmed_word == stem_word:
                case_list.append(cat)
    return case_list

example_word = 'дом' # input('Введите неименяемую часть слова. Например "дом", "квартир", "автомобил" и т.д')
category(example_word)

['сгорел дом', 'замкнуло электричество дома', 'затопило дом']

**Категоризация данных**

Cловарь — распространенный способ хранения информации, который помогает убрать текстовые параметры из основной таблицы и увеличить эффективность работы с данными. Выделим словарь для возрастных групп.

In [641]:
age_dict = data[['age_group', 'age_id']].drop_duplicates().sort_values(by='age_id')
age_dict

,age_group,age_id
4,молодой,2
2,зрелый,3
0,пожилой,4


Категоризируем причины выплаты страховых платежей с помощью лемматизации.

In [644]:
m = Mystem()

def lem(row):
    lemmas = m.lemmatize(row)
    print(lemmas)
    if 'авто' in lemmas or 'дтп' in lemmas or 'автомобиль' in lemmas: return 'Случаи с автомобилями'
    elif 'отдых' in lemmas or 'отпуск' in lemmas: return 'Случаи во время отпуска'
    elif 'дом' in lemmas or 'дача' in lemmas or 'квартира' in lemmas or 'жилье' in lemmas: return 'Случаи с недвижимостью'
        
data['categ_cause'] = data['cause'].apply(lem)

['повреждение', ' ', 'автомобиль', '\n']
['упасть', ' ', 'дерево', ' ', 'на', ' ', 'автомобиль', '\n']
['сломать', ' ', 'нога', ' ', 'в', ' ', 'отпуск', '\n']
['хищение', ' ', 'автомобиль', ' (', 'каско', ')\n']
['несчастный', ' ', 'случай', ' ', 'в', ' ', 'отпуск', '\n']
['врезаться', ' ', 'в', ' ', 'автомобиль', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['сгорать', ' ', 'квартира', '\n']
['сгорать', ' ', 'дом', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['кража', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['потоп', ' ', 'в', ' ', 'квартира', '\n']
['украсть', ' ', 'багаж', ' ', 'в', ' ', 'отпуск', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['травма', ' ', 'во', ' ', 'время', ' ', 'отдых', '\n']
['повреждение', ' ', 'авто', '\n']
['проникать', ' ', 'в', ' ', 'жилье', '\n']
['повреждение', ' ', 'автомобиль', '\n']
['украсть', ' ', 'авто', '\n']
['сгорать', ' ', 'дом', '\n']
[

Категоризируем статус выплаты компенсации — 0 выплат, выплачена часть, полностью выплачена.

In [647]:
def pay_stat(pay):
    if pay == 1: return 'Полностью выплачено'
    elif pay == 0: return '0 выплачено'
    else: return 'Выплачена часть'

data['paid'] = data['perc_of_compensation_paid'].apply(pay_stat)

Категоризируем `income`, разбив данные по квантилям.

In [648]:
data['quant'] = pd.qcut(data['income'], 5)

## Результаты

Подготовим 3 таблицы и изучим результат.

*ответьте на вопрос — есть ли зависимость между возрастной группой и статусом выплаты?*

In [649]:
pd.pivot_table(data, index='age_group', columns='paid', values='id', aggfunc='count')

paid,0 выплачено,Выплачена часть,Полностью выплачено
age_group,,,
зрелый,62,884,100
молодой,31,206,52
пожилой,57,698,204


*Глобальной зависимости между возрастной группой и статусом выплаты нету. Например, пожилым людям выплачено полностью больше, чем, скажем, зрелым, однако у вторых частичная выплата составляет большую часть, чем у пожилых.*

*есть ли зависимость между причиной страхового случая и статусом выплаты?*

In [650]:
pd.pivot_table(data, index='categ_cause', columns='paid', values='id', aggfunc='count')

paid,0 выплачено,Выплачена часть,Полностью выплачено
categ_cause,,,
Случаи во время отпуска,42,536,78
Случаи с автомобилями,54,640,182
Случаи с недвижимостью,54,612,96


*Большой разницы между причиной страхового случая и статусом выплаты нету, однако со статусом "Польностью выплачено" больше случаев, связанных с автомобилями, чем во время отпуска или с недвижимостью.*

*есть ли зависимость между доходом клиента и статусом выплаты?*

In [651]:
pd.pivot_table(data, index='quant', columns='paid', values='id', aggfunc='count')

paid,0 выплачено,Выплачена часть,Полностью выплачено
quant,,,
"(-0.001, 90060.0]",43,326,94
"(90060.0, 135758.0]",30,356,69
"(135758.0, 189668.0]",33,366,59
"(189668.0, 269868.0]",19,371,69
"(269868.0, 2700040.0]",25,369,65


*Зависимости между доходом клиента и статусом выплаты нету. Разница между всеми выплатами несущественная.*